In [1]:
!python --version
!nvcc --version
!pip install nvcc4jupyter
%load_ext nvcc4jupyter

Python 3.10.12
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmpytz1zwwk".


In [5]:
%%cuda
#include <stdio.h>
#include<stdio.h>
#include<stdlib.h>

#define SHARED_BLOCK_SIZE 64

struct GpuTimer {
      cudaEvent_t start;
      cudaEvent_t stop;

      GpuTimer()
      {
            cudaEventCreate(&start);
            cudaEventCreate(&stop);
      }

      ~GpuTimer()
      {
            cudaEventDestroy(start);
            cudaEventDestroy(stop);
      }

      void Start()
      {
            cudaEventRecord(start, 0);
      }

      void Stop()
      {
            cudaEventRecord(stop, 0);
      }

      float Elapsed()
      {
            float elapsed;
            cudaEventSynchronize(stop);
            cudaEventElapsedTime(&elapsed, start, stop);
            return elapsed;
      }
};



__global__ void matrix_transpose_naive(int *input, int *output, int n) {

	int indexX = threadIdx.x + blockIdx.x * blockDim.x;
	int indexY = threadIdx.y + blockIdx.y * blockDim.y;
	int index = indexY * n + indexX;
	int transposedIndex = indexX * n + indexY;

    // this has discoalesced global memory store
	output[transposedIndex] = input[index];

	// this has discoalesced global memore load
	// output[index] = input[transposedIndex];
}

__global__ void matrix_transpose_shared(int *input, int *output, int n) {

	__shared__ int sharedMemory [SHARED_BLOCK_SIZE] [SHARED_BLOCK_SIZE];

	// global index
	int indexX = threadIdx.x + blockIdx.x * blockDim.x;
	int indexY = threadIdx.y + blockIdx.y * blockDim.y;

	// transposed global memory index
	int tindexX = threadIdx.x + blockIdx.y * blockDim.x;
	int tindexY = threadIdx.y + blockIdx.x * blockDim.y;

	// local index
	int localIndexX = threadIdx.x;
	int localIndexY = threadIdx.y;

	int index = indexY * n + indexX;
	int transposedIndex = tindexY * n + tindexX;

	// reading from global memory in coalesed manner and performing tanspose in shared memory
	sharedMemory[localIndexX][localIndexY] = input[index];

	__syncthreads();

	// writing into global memory in coalesed fashion via transposed data in shared memory
	output[transposedIndex] = sharedMemory[localIndexY][localIndexX];
}

void fill_array(int *data, int n) {
	for(int idx=0;idx<(n*n);idx++)
		data[idx] = idx;
}



int main(void) {
  int sizes[] = {512, 1024, 2048, 4096, 8192, 16384};
	int blocks[] = {8, 16, 32, 64};
	int *a, *b;
  int *d_a, *d_b;

  printf("n;type;block_size;elapsed_time\n");


  for(int s = 0; s < sizeof(sizes) / sizeof(sizes[0]); s++) {
        int n = sizes[s];
        int size = n * n * sizeof(int);

        for (int t = 0; t < sizeof(blocks) / sizeof(blocks[0]); t++) {

            a = (int *)malloc(size); fill_array(a, n);
            b = (int *)malloc(size);


            cudaMalloc((void **)&d_a, size);
            cudaMalloc((void **)&d_b, size);

            cudaMemcpy(d_a, a, size, cudaMemcpyHostToDevice);
            cudaMemcpy(d_b, b, size, cudaMemcpyHostToDevice);


            GpuTimer timer;

            int block_size = blocks[t];

            dim3 blockSize(block_size,block_size,1);
            dim3 gridSize(n/block_size,n/block_size,1);

            timer.Start();
            matrix_transpose_naive<<<gridSize,blockSize>>>(d_a,d_b, n);
            cudaDeviceSynchronize();
            timer.Stop();
            printf("%d;naive;%d;;%f\n",size,block_size,timer.Elapsed());

            free(a); free(b);
            cudaFree(d_a); cudaFree(d_b);
        }

    }


  for(int s = 0; s < sizeof(sizes) / sizeof(sizes[0]); s++) {
        int n = sizes[s];
        int size = n * n * sizeof(int);

        for (int t = 0; t < sizeof(blocks) / sizeof(blocks[0]); t++) {

            a = (int *)malloc(size); fill_array(a, n);
            b = (int *)malloc(size);


            cudaMalloc((void **)&d_a, size);
            cudaMalloc((void **)&d_b, size);

            cudaMemcpy(d_a, a, size, cudaMemcpyHostToDevice);
            cudaMemcpy(d_b, b, size, cudaMemcpyHostToDevice);


            GpuTimer timer;

            int block_size = blocks[t];

            dim3 blockSize(block_size,block_size,1);
            dim3 gridSize(n/block_size,n/block_size,1);

            timer.Start();
            matrix_transpose_shared<<<gridSize,blockSize>>>(d_a,d_b, n);
            cudaDeviceSynchronize();
            timer.Stop();
            printf("%d;shared;%d;%f\n",n,block_size,timer.Elapsed());

            free(a); free(b);
            cudaFree(d_a); cudaFree(d_b);
        }

    }


	return 0;
}


n;type;block_size;elapsed_time
1048576;naive;8;;0.240448
1048576;naive;16;;0.036640
1048576;naive;32;;0.062944
1048576;naive;64;;0.008672
4194304;naive;8;;0.065856
4194304;naive;16;;0.088256
4194304;naive;32;;0.200672
4194304;naive;64;;0.008832
16777216;naive;8;;0.234144
16777216;naive;16;;0.314784
16777216;naive;32;;0.763456
16777216;naive;64;;0.010176
67108864;naive;8;;0.945408
67108864;naive;16;;1.246368
67108864;naive;32;;2.935136
67108864;naive;64;;0.009344
268435456;naive;8;;6.120992
268435456;naive;16;;5.703744
268435456;naive;32;;11.725920
268435456;naive;64;;0.008288
1073741824;naive;8;;24.947041
1073741824;naive;16;;25.917408
1073741824;naive;32;;46.681728
1073741824;naive;64;;0.008576
512;shared;8;0.044448
512;shared;16;0.035104
512;shared;32;0.043072
512;shared;64;1.058688
1024;shared;8;0.125120
1024;shared;16;0.078784
1024;shared;32;0.106304
1024;shared;64;1.058944
2048;shared;8;0.451008
2048;shared;16;0.247808
2048;shared;32;0.310848
2048;shared;64;0.007776
4096;shared;8;